In [1]:
import pandas as pd

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import airtable_utils as a
import pandas_utils as p
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()

rerun_query = 1

at_base_id = os.environ["AIRTABLE_DEVREL_BASE_ID"]

at_table_name = 'OP Project Data'

excl_projects = [
    '',
    # COMMENT OUT TO KEEP ALL PROJECTS
    # 'Op',
    # 'Optimism Governor'
]

In [2]:
query_id = 2709489 #https://dune.com/queries/2709489
name = "Project Usage Trends - T28D"

if rerun_query == 0:
    projects = pd.read_csv('csv_outputs/' + name + '.csv')
else:
    # Dune query: https://dune.com/queries/2457627
    projects = d.get_dune_data(query_id, name = name)

In [3]:
projects = projects[~projects['Team'].isin(excl_projects)]

In [4]:
#reformat col names
projects = p.csv_columns_to_formatted(projects)

datestr = p.get_datestring_from_datetime(projects['Date'].iloc[0])
datestr = datestr.replace('-', '_')

p.mkdir_if_not_exists('uploads')
# deployers.to_csv('uploads/deployer_data_' + datestr + '.csv', index=False)

In [5]:
print(projects.columns)

Index(['Team', 'Address Type', 'Date',
       'Total Calldata Gas To W Overhead Growth',
       'Total Calldata Gas To W Overhead T28D',
       'Total Eth Fees Split By Trace Gas Growth',
       'Total Eth Fees Split By Trace Gas T28D', 'Total Eth Fees To Growth',
       'Total Eth Fees To T28D', 'Total Num Txs To Growth',
       'Total Num Txs To T28D', 'Total Num Txs Trace Growth',
       'Total Num Txs Trace T28D', 'Total Trace Level Eth Fees Growth',
       'Total Trace Level Eth Fees T28D', 'Total Trace Level Gas Used Growth',
       'Total Trace Level Gas Used T28D', 'Total Trace Tx Eth Fees Growth',
       'Total Trace Tx Eth Fees T28D', 'Last Updated'],
      dtype='object')


In [6]:
# Delete Existing Data
# print('deleting existing data')S
a.delete_all_records(at_base_id,at_table_name)
# Append New Data
print('appending new data')
# deployers = deployers[deployers['Team'] == 'Cozy Finance'] #Test specific team
a.update_database(at_base_id,at_table_name, projects)
print('done')

0
All records have been deleted
appending new data
The base 'appQ2S9IatldkoNQw' already exists, appending new records...
['recn6ECl56j7EFZoR']
add new record
add new record
['recWNKnIQuV4bWVY3', 'recxTLp28rvXVccAy']
add new record
['recEzL5mSjKrovQV7']
add new record
['rectFNHOnvcR4OZEo']
add new record
['rec2HLV86zow1VG6E']
add new record
['reccwb2zCIEfS04bN', 'recZcARuzhYO5KXjh']
add new record
['rec391yw7FIIDuXK6']
add new record
['recjBBbBIun05qfqn']
add new record
['recssvVmx1Kkwz4ca']
add new record
['recJUH4glpBmgxpZS']
add new record
['rec1G2derj03tZLW9', 'rec1Dyp7QBshFe6Mz']
add new record
['recNIXYemRk12CUWR']
add new record
['recOXlrN5Wnh0hr5k']
add new record
['reccHOvuENoFd688Q', 'recmBjok4PU9vQaDG']
add new record
['recNkg2CKrJhV0rqP']
add new record
['recS7mmHc6sSpf0Au']
add new record
['reckzZLxmCL7tHkvm']
add new record
['rec1zgtjWcHAw0ivX']
add new record
['recEhKgMbVkoKiar4']
add new record
['rec8ZIzt3ZnAMRsaS']
add new record
['rec8p0DdOY1li4JuA', 'recDRwrhfEkgFg6aH